In [7]:
# # #Initializing the parameters
# # Read about how a Parameter cell should be used for defining and initializing parameters in Synapse

# Storage Account Name
StorageAccountName = ""
# Main container/directory on the storage account
VivaInsightsDataFileSystem = ""

PipelineId = ""
PersonQueryDatasetFolder = ""
SecondaryEmployeeId = ""

# Database connection information
SQLServerEndpoint = ""
DBName = ""
DBUser = ""
DBPass = ""
DBPort = ""



StatementMeta(SparkPool2, 41, 7, Finished, Available)

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import DateType


# constants_ path template to access storage account for read and write
# inputFilePath = "abfss://{}@{}.dfs.core.windows.net/{}/raw/{}/*.txt"

# storageAccount = "{}.dfs.core.windows.net"
# # outputFilePath = "https://{}.dfs.core.windows.net/{}/{}"
# outputFilePath = "https://%s.dfs.core.windows.net/%s/%s"

#Setting Prameters
# extractionFS = VivaInsightsDataFileSystem


print("PipelineId is: ", PipelineId)

StatementMeta(SparkPool2, 41, 8, Finished, Available)

PipelineId is:  6f714f55-1b3c-4a32-ad47-deb4a3741f3f

In [9]:
#Reading meeting csv file from storage account
inputFilePath = 'abfss://{0}@{1}.dfs.core.windows.net/{2}/raw/{3}/*.txt'.format(VivaInsightsDataFileSystem, StorageAccountName, PipelineId, PersonQueryDatasetFolder)
personDf = spark.read.csv(inputFilePath, header = 'true', inferSchema= 'true')

# Dataframe prep
personDf = personDf.withColumn("Date",personDf['Date'].cast(DateType()))
personDf = personDf.withColumnRenamed(SecondaryEmployeeId,"EmployeeId")

# personDf.printSchema()
personDf.createOrReplaceTempView('personDf')

StatementMeta(SparkPool2, 41, 9, Finished, Available)

In [10]:
# # Checking the Database for the last existing record
jdbcHostname = SQLServerEndpoint
jdbcDatabase = DBName
jdbcPort = DBPort

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
   "user" : DBUser,
   "password" : DBPass,
   "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
pushdown_query = "(Select max(Date) as temp from viva_insights_person) tempTbl"
latestExistingDate = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties).first().temp
print("Latest existing person date in DB is", latestExistingDate)


StatementMeta(SparkPool2, 41, 10, Finished, Available)

Latest existing person date in DB is 2021-11-07

In [11]:
# Preparing dataframe for upsert/insert into database
# # Record selection
if (latestExistingDate == None):
    outputStatus = "FullUpload"
    outputDf = personDf
else:
    outputStatus = "PartialUpload"
    outputDf = personDf[personDf.Date > latestExistingDate]


# Attribute selection

columns = ["PersonId","EmployeeId", "Date", "Organization", "LevelDesignation", "Workweek_span", "Meetings_with_skip_level", "Meeting_hours_with_skip_level"
, "Generated_workload_email_hours", "Generated_workload_email_recipients", "Generated_workload_instant_messages_hours"
, "Generated_workload_instant_messages_recipients", "Generated_reactions_to_posts", "Generated_replies_to_posts"
, "Generated_workload_call_hours", "Generated_workload_call_participants", "Generated_workload_calls_organized"
, "External_network_size", "Internal_network_size", "Networking_outside_company", "Networking_outside_organization"
, "Multitasking_hours", "After_hours_meeting_hours", "Open_1_hour_block", "Open_2_hour_blocks", "Total_focus_hours"
, "Low_quality_meeting_hours", "Meetings", "Meeting_hours", "Conflicting_meeting_hours", "Multitasking_meeting_hours"
, "Redundant_meeting_hours__lower_level_", "Redundant_meeting_hours__organizational_"
, "Time_in_self_organized_meetings", "Meeting_hours_during_working_hours", "Generated_workload_meeting_attendees"
, "Generated_workload_meeting_hours", "Generated_workload_meetings_organized", "Manager_coaching_hours_1_on_1"
, "Meetings_with_manager", "Meeting_hours_with_manager", "Meetings_with_manager_1_on_1"
, "Meeting_hours_with_manager_1_on_1", "After_hours_instant_messages", "Instant_messages_sent", "Instant_Message_hours"
, "Working_hours_instant_messages", "Emails_sent", "Email_hours", "Uninterrupted_focus_hours"
, "After_hours_collaboration_hours", "Collaboration_hours_external", "Collaboration_hours"
, "Working_hours_collaboration_hours", "After_hours_email_hours", "Working_hours_email_hours"
, "Channels_with_active_engagement", "Teams_with_active_engagement", "After_hours_channel_message_hours"
, "Channel_message_hours", "Channel_messages_sent", "Channel_reactions", "Channel_visits"
, "Working_hours_channel_message_hours", "After_hours_in_calls", "Total_calls", "Call_hours"
, "Working_hours_in_calls", "IsInternal", "IsActive", "WorkingStartTimeSetInOutlook", "WorkingEndTimeSetInOutlook"
, "WorkingDaysSetInOutlook"]

outputDf = outputDf.select([col for col in columns])

# display(outputDf)
print("OutputStatus is: ", outputStatus)
print("Number of records inserted is: ", outputDf.count())



StatementMeta(SparkPool2, 41, 11, Finished, Available)

OutputStatus is:  PartialUpload
Number of records inserted is:  0

In [12]:
# Insert/Upsert into database

mode = "append"
outputDf.write.jdbc(url=jdbcUrl, table="dbo.viva_insights_person", mode=mode, properties=connectionProperties)

StatementMeta(SparkPool2, 41, 12, Finished, Available)